In [2]:
# 📦 SECTION 1: Imports and Configuration
import pynwb
import h5py
import remfile
from dandi.dandiapi import DandiAPIClient
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
# Download and read NWB files from a Dandiset, one file per subject
client = DandiAPIClient()
dandiset = client.get_dandiset("000253", "draft")

# Dictionary to store units tables from each file
units_tables = {}
file_metadata = {}

# Track subjects we've already processed
processed_subjects = set()

# Iterate over all assets in the dandiset
for asset in dandiset.get_assets():
    # Extract subject ID from path (assuming format like "sub-632485/...")
    path_parts = asset.path.split('/')
    if len(path_parts) > 0 and path_parts[0].startswith('sub-'):
        subject_id = path_parts[0]
        
        # Skip if we've already processed this subject
        if subject_id in processed_subjects:
            continue
            
        print(f"Processing file: {asset.path} (subject: {subject_id})")
        
        try:
            url = asset.download_url
            remote_file = remfile.File(url)
            h5_file = h5py.File(remote_file)
            io = pynwb.NWBHDF5IO(file=h5_file, load_namespaces=True)
            nwb = io.read()
            
            # Check if the file contains units
            if hasattr(nwb, 'units') and nwb.units is not None and len(nwb.units) > 0:
                units = nwb.units
                print(f"  Found units table with {len(units)} units")
                print(f"  Column names: {units.colnames}")
                
                # Create a unique key for this file using subject ID
                file_key = subject_id
                
                # Store the units table for this file
                units_tables[file_key] = units
                
                # Store metadata about this file
                file_metadata[file_key] = {
                    'path': asset.path,
                    'n_units': len(units),
                    'columns': units.colnames,
                    'subject_id': nwb.subject.subject_id if hasattr(nwb, 'subject') and nwb.subject else None,
                    'session_start_time': nwb.session_start_time if hasattr(nwb, 'session_start_time') else None
                }
                
                # # Example: Access units from specific file
                # if 'firing_rate' in units.colnames:
                #     firing_rates = units['firing_rate'][:]
                #     print(f"  Mean firing rate: {firing_rates.mean():.2f} Hz")
                
                # Mark this subject as processed
                processed_subjects.add(subject_id)
                
            else:
                print(f"  Skipping - no units table found or empty")
                
            # Clean up
            io.close()
            h5_file.close()
            
        except Exception as e:
            print(f"  Error processing {asset.path}: {e}")
            continue
    else:
        print(f"Skipping {asset.path} - doesn't match subject pattern")

print(f"\nProcessed {len(units_tables)} subjects with units tables")
print(f"Available subjects: {list(units_tables.keys())}")

Processing file: sub-621890/sub-621890_ses-1186358749_ogen.nwb (subject: sub-621890)
  Found units table with 2446 units
  Column names: ('cumulative_drift', 'local_index', 'firing_rate', 'l_ratio', 'silhouette_score', 'spread', 'cluster_id', 'velocity_above', 'recovery_slope', 'nn_miss_rate', 'nn_hit_rate', 'waveform_halfwidth', 'quality', 'peak_channel_id', 'isi_violations', 'amplitude_cutoff', 'd_prime', 'repolarization_slope', 'PT_ratio', 'snr', 'isolation_distance', 'amplitude', 'max_drift', 'waveform_duration', 'presence_ratio', 'velocity_below', 'spike_times', 'spike_amplitudes', 'waveform_mean')
Processing file: sub-632487/sub-632487_ses-1204677304_ogen.nwb (subject: sub-632487)
  Found units table with 2745 units
  Column names: ('nn_miss_rate', 'velocity_above', 'max_drift', 'waveform_duration', 'silhouette_score', 'l_ratio', 'snr', 'cumulative_drift', 'isi_violations', 'recovery_slope', 'peak_channel_id', 'local_index', 'waveform_halfwidth', 'amplitude', 'PT_ratio', 'velocit

### The cell below is for the final script to download all the units from the groundtruth dateset. It should not be executed currently, so should be kept as commented out.

In [ ]:
# # HIPPIE-Compatible Unit Export from DANDI Dataset 000253

# # 📦 SECTION 1: Imports and Configuration
# import os
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from dandi.dandiapi import DandiAPIClient
# from pynwb import NWBHDF5IO
# from pathlib import Path

# # 📁 Create local download path
# dandi_id = '000253'
# download_path = f'./dandi_{dandi_id}_nwb/'
# os.makedirs(download_path, exist_ok=True)

# # 🔽 SECTION 2: Download NWB Files from DANDI
# with DandiAPIClient() as client:
#     assets = client.get_dandiset(dandi_id, 'draft').get_assets()
#     for asset in assets:
#         asset.download(download_path)

# # 📂 SECTION 3: Load NWB Files and Extract Units
# nwb_files = list(Path(download_path).glob("*.nwb"))
# unit_data = []

# for file in nwb_files:
#     with NWBHDF5IO(file, 'r') as io:
#         nwbfile = io.read()
#         if nwbfile.units is None:
#             continue
#         units_table = nwbfile.units.to_dataframe()
#         unit_data.append((file.name, nwbfile, units_table))

# # 📈 SECTION 4a: Extract Average Waveforms
# waveform_rows = []

# for file_name, nwbfile, units_df in unit_data:
#     for unit_idx, row in units_df.iterrows():
#         waveforms = row.get('waveforms', None)
#         if waveforms is not None and isinstance(waveforms, np.ndarray):
#             mean_wave = np.mean(waveforms, axis=0)
#             waveform_rows.append([unit_idx] + list(mean_wave))

# waveform_df = pd.DataFrame(waveform_rows)
# if not waveform_df.empty:
#     waveform_df.columns = ['unit_id'] + [f'chan_{i}' for i in range(waveform_df.shape[1] - 1)]
#     waveform_df.to_csv('waveforms.csv', index=False)
# else:
#     print("⚠️ No waveform data found.")

# # ⏱ SECTION 4b: Extract ISI Histogram per Unit
# isi_rows = []

# for _, _, units_df in unit_data:
#     for unit_idx, row in units_df.iterrows():
#         spike_times = row['spike_times']
#         if isinstance(spike_times, np.ndarray) and len(spike_times) > 2:
#             isis = np.diff(spike_times)
#             hist, _ = np.histogram(isis, bins=50, range=(0, 1.0))
#             isi_rows.append([unit_idx] + list(hist))

# isi_df = pd.DataFrame(isi_rows)
# if not isi_df.empty:
#     isi_df.columns = ['unit_id'] + [f'b{i}' for i in range(50)]
#     isi_df.to_csv('isis.csv', index=False)
# else:
#     print("⚠️ No ISI data found.")

# # 🧾 SECTION 4c: Extract Unit Metadata
# meta_rows = []

# for file_name, nwbfile, units_df in unit_data:
#     for unit_idx, row in units_df.iterrows():
#         meta_rows.append({
#             'unit_id': unit_idx,
#             'source_file': file_name,
#             'location': row.get('location', 'unknown'),
#             'electrode_group': row.get('electrode_group', 'unknown'),
#         })

# metadata_df = pd.DataFrame(meta_rows)
# if not metadata_df.empty:
#     metadata_df.to_csv('metadata.csv', index=False)
# else:
#     print("⚠️ No metadata found.")

# # ✅ SECTION 5: Sanity Checks (Optional)
# if not waveform_df.empty:
#     print("waveforms.csv shape:", waveform_df.shape)
# if not isi_df.empty:
#     print("isis.csv shape:", isi_df.shape)
# if not metadata_df.empty:
#     print("metadata.csv shape:", metadata_df.shape)

# # Example plot of ISI histogram
# if not isi_df.empty:
#     isi_df.iloc[0, 1:].plot(kind='bar', title=f"ISI Histogram for Unit {isi_df.iloc[0]['unit_id']}")
#     plt.xlabel("ISI Bin")
#     plt.ylabel("Count")
#     plt.tight_layout()
#     plt.show()
# else:
#     print("⚠️ Skipping plot: no ISI data to display.")
